# Мотивация (или зачем это всё надо)

Это задание достаточно объемное и будет стоить 20 баллов, однако это совсем не главное и интересное в нём. 

Что действительно круто - оно даёт вам возможность немножко прикоснуться к совершенно другому миру, к потрясающим методам и моделям, которые могут предсказывать будущее, ставить диагнозы, выявлять раковые опухоли, говорить, какое же вино самое крутое на вкус и многое-многое другое. 

Небольшое лирическое отступление и немножко моего мнения, с которым, если хочется, можно и не соглашаться. На мой взгляд, огромная проблема системы образования - это наличие оценок. Погоня за ними может вытеснить изначальное любопытство и желание экспериментировать, потому что страх "не сдать" будет сильнее. Но у нас тут с вами знакомство с дивным новым миром, со штуками, которые проникают во все сферы жизни и активно на них влияют (как минимум [юристам в Сбербанке](https://www.rbc.ru/business/23/07/2017/5974b7a69a79477896b6708d) из-за них уже живется несладко).

Поэтому мне бы очень хотелось, если бы этот проект вы воспринимали именно как возможность попробовать что-то новое для себя, пройти через все ~~круги ада~~ стадии работы с данными, и получить в финале свой крутой "data science pet-project". Если вдруг вы когда-нибудь решите углубляться в это дело - у вас будет очень и очень хороший старт, более того, такие проекты иногда дают в качестве тестового задания при устройстве на работу аналитиком. Если же нет - вы будете очень хорошо представлять, сколько боли, слёз и вылетающих ошибок ждёт аналитика, когда ему падает такое задание ;) 

Так что дерзайте, оценивать тут буду чуть более субъективно, все-таки у всех датасеты сильно разные, походы и методы будут тоже различаться. Но в общем и целом для успешной сдачи достаточно сделать два шага:
1. Получить датасет с признаками и целевой переменной, где все значения будут числовыми, а пропусков не будет совсем
2. Построить на обучающей части две модели, сделать прогноз на тестовой и померять качество каждой

Поехали

![](https://media.giphy.com/media/b7f0X8Okk1uyk/source.gif)

# Предобработка данных и построение моделей

# Предобработка

На этом этапе необходимо основательно поработать с переменными, имеющимися в датасете, чтобы привести их все к числовому виду, очистить от пропусков, выкинуть те переменные, где пропусков слишком много (>50%).

Чтобы посмотреть, в каких переменных пропуски есть, можно воспользоваться

```python
data.isnull().sum()
```

А чтобы получить процент пропусков, достаточно поделить на длину датасета

```python
data.isnull().sum()/len(data)
```


Также нужно убедиться, что никаких переменных, явно связанных с целевой, не осталось в наборе признаков X - например: 

- если вы предсказываете жив или мертв персонаж из Игры Престолов, лучше выкинуть их датасета имеющиеся вероятности их жизни/смерти, 
- если предсказываете суммарно число арендованных велосипедов, стоит выкинуть число зарегистрированных и незарегистрированных велосипедов, которые взяты в аренду в этот день,
- если предсказываете победу/поражение футбольной команды, выкидываем информацию о забитых голах в течение этого матча
- и т.д.

Чтобы выкинуть переменную необходимо воспользоваться одним из предложенных вариантов команд:

```python
data.drop(['название_переменной'], axis=1, inplace=True)

data = data.drop(['название_переменной'], axis=1)
```


## Непрерывные переменные

Скорее всего непрерывные переменные будут с какими-то пропусками. Чтобы  модели могли все-таки с этими переменными работать, нужно эти пропуски заполнить. Мы будем заполнять средними или медианами (на ваш выбор), так как это достаточно "нейтральные" значения, которые ничего нового моделям не сообщат, но будут служить хорошим заполнителем. 

Чтобы заполнить пропущенное значение средним/медианой можно сделать одно из следующих действий:

```python
data['название_переменной'].fillna(data['название_переменной'].mean(), inplace = True)

data['название_переменной'] = data['название_переменной'].fillna(data['название_переменной'].mean())
```

В случае, если у вас задача регрессии (целевая переменная является непрерывной), имеет смысл посмотреть, какое у неё распределение - если оно сильно скошено вправо и имеет тяжелый правый хвост, будет крайне полезно её прологарифмировать, как мы это делали на семинаре. После построения моделей и получения предсказаний, не забудте сделать обратное действие - возведение в экспоненту.

```python
data['непрерывная_целевая'] = np.log(data['непрерывная_целевая'])

y_pred = np.exp(y_pred_log)
```

## Категориальные переменные

Здесь уже повеселее. Начнем с заполнения пропусков - правило точно такое же, если  больше 50% пропусков, выкидываем напрочь, если меньше - будем заполнять. Обычно пропущенные категориальные значения заполняются самым часто встречающимся значением (по сути - наиболее вероятным). То есть если у вас в выборке 90% кошечек, 9% собачек и 1% пропусков - этот 1% можно смело заполнять кошечками. 

```python
# после проверки процента пропусков, смотрим на самый часто встречающийся класс
data['категориальная_переменная'].value_counts().head(1)

# допустим, самое часто встречающееся - "кошечка", заполняем пропуски им
data['категориальная_переменная'].fillna("кошечка", inplace = True)
```


Категориальные переменные у вас могут быть самые разные, от бинарных (мужчина/женщина) до каких-нибудь стран/городов с сотнями уникальных значений. Давайте договоримся, что если уникальных значений категориальной переменной больше 20, то мы не делаем с ней  One Hot Encoding, а вместо этого кодируем её при помощи частот.

Итак, для начала стоит проверить, а сколько же у вас уникальных значений у категориальной переменной:

```python
len(data['категориальная_переменная'].unique())
```

Если эта штука меньше, либо равна 20, делаем OHE, как на семинарах:

```python
# кодируем переменную и выкидываем первый столбец
dummies = pd.get_dummies(data['категориальная_переменная'], prefix='название_будущих_столбцов', drop_first=True)

# добавляем закодированные значения в наш исходный датасет
data = pd.concat([data, dummies], axis=1)

# выкидываем изначальную переменную, так как мы её закодировали другими
data.drop(['категориальная_переменная'], axis=1, inplace=True)
```

Если же у нас больше 20 уникальных категорий, то сделаем другое преобразование, чтобы не так сильно раздувать датасет, а именно - кодирование частотой. То есть мы посчитаем, как часто встречается каждая категория в датасете и эту частоту подставим вместо самой категории. Тогда при помощи такого небольшого хака у нас получится столбец с непрерывной переменной вместо категориальной.

```python
# считаем частоты для каждого возможного значения переменной
frequency_encoding = dict(data['категориальная_переменная'].value_counts()/len(data))

# преобразовываем категориальную переменную в частоты
data['категориальная_переменная'] = data['категориальная_переменная'].map(frequency_encoding)
```

## Что делать, если переменных очень-очень-очень много (больше 20)

Если так сильно повезло с датасетом, что переменных завались и разгребать каждую сил нет - во-первых, дропаем всё, где много пропусков, можно даже дропать те, где пропусков больше 30%, во-вторых, можно изначально посмотреть на столбцы и прикинуть, что из этого может быть полезно при предсказании, а что нет, и выбрать штук 10 только для анализа и моделирования, для этого задания будет достаточно. 

---

# Моделирование

Здесь всё достаточно просто, если вам удалось получить датасет, в котором одни сплошные числа и пропусков нет, считайте, что всё готово. На этом этапе необходимо сделать три вещи:

1. разбить  выборку на обучающую и тестовую (оставить для теста 30% датасета)
2. построить на обучающей выборке модели, посмотреть на важность признаков, сделать предсказания на тестовой
3. по разным метрикам качества оценить, хорошо ли модель напрогнозировала

А теперь подробнее по каждому пункту:

## Разбиение выборки
Вот вы получили себе датасет, в котором есть какие-то признаки и целевая переменная. Давайте назовём целевую переменную $y$ для удобства. Для того, чтобы разбить выборку достаточно сделать следующее:

```python
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size = 0.3)

y_train = df_train['y']
y_test = df_test['y']

X_train = df_train.drop(['y'], axis=1)
X_test = df_test.drop(['y'], axis=1)
```

Готово, вы великолепны. 

## Построение моделей - Регрессия

Здесь идем прямо по семинару и делаем те же самые шаги - шкалируем непрерывные переменные при помощи StandardScaler, учим линейную регрессию и случайный лес, смотрим на важность признаков, делаем предсказания и смотрим на качество. 

## Построение моделей - Классификация

Здесь тоже придется построить две модели, одну из них вы знаете - это случайный лес, а вторая - логистическая регрессия, это такая специальная регрессия, которая умеет предсказывать вероятности отнесения объектов к тому или иному классу. Строится она точно так же просто, как и линейная регрессия:

```python
from sklearn.linear_model import LogisticRegression
# объявляем модель
logit = LogisticRegression()
# обучаем на тренировочной выборке
logit.fit(X_train, y_train)
# предсказываем вероятности
logit.predict_proba(X_test)
# предсказываем сами метки классов
logit.predict(X_test)
```

Единственное замечаение - если у вас датасет практически не требовал предобработки (все данные уже числовые или буквально пару манипуляций надо было сделать), то придется постараться и сделать дополнительно настройку параметров для случайного леса - подобрать при помощи GridSearch оптимальное число деревьев.

---
# Метрики качества

## Регрессия

Здесь всё просто, используем все те метрики, что разобрали на семинаре:
```python
print('MAE:', metrics.mean_absolute_error(np.exp(y_test), np.exp(y_pred)))
print('RMSE:', np.sqrt(metrics.mean_squared_error(np.exp(y_test), np.exp(y_pred))))
print('R2:',  metrics.r2_score(y_test, y_pred))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred))
```

## Бинарная классификация (два класса)
В этом случае используем весь наш зоопарк метрик с семинара, при этом не забываем, что для рассчета площади под ROC кривой нужно использовать именно предсказанные вероятности, а не сами метки:
```python
print('Accuracy =', metrics.accuracy_score(y_test, y_predicted))
print('Precision =', metrics.precision_score(y_test, y_predicted))
print('Recall =', metrics.recall_score(y_test, y_predicted))
print('F1_score =', metrics.f1_score(y_test, y_predicted))
print('AUC =', metrics.roc_auc_score(y_test, y_proba[:,1]))
```

## Многоклассовая классификация (три и более классов)
В этом случае будем использовать очень удобную штуку под названием classification_report, где автоматически считаются precision и recall для каждого класса в отдельности вместе с f1-метрикой, а затем еще и среднее по ним по всем:

```python
print(metrics.classification_report(y_test, y_predicted))
```

На этом всё, дедлайн по заданию будет 16 июня 23:59, чтобы точно успел всё проверить до экзамена.

По всем вопросам - как обычно пишите, будем разбираться :)